In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import urllib.request, json 
import pandas as pd
from datetime import datetime, timezone
import time
from dateutil.parser import parse

In [24]:
file_list = [f"2020-1-{j}_df_all_features_winner.csv" for j in [i for i in range(5,7)]]
#df2 = pd.read_csv("2020-1-5_df_all_features_winner.csv")
#print(file_list)
#print(df2)
df2  = pd.DataFrame()#columns = [i for i in range(30)])
for k in file_list:
    df2 = df2.append(pd.read_csv(k), sort = True)

In [25]:
print(df2.columns)
print(df2)

Index(['Away_OT', 'Away_blocked', 'Away_faceOffWinPercentage',
       'Away_giveaways', 'Away_goals', 'Away_hits', 'Away_losses', 'Away_pim',
       'Away_powerPlayGoals', 'Away_powerPlayOpportunities',
       'Away_powerPlayPercentage', 'Away_shots', 'Away_takeaways', 'Away_wins',
       'Blocked_Diff', 'Giveaways_Diff', 'Goals_Diff', 'Hits_Diff', 'Home_OT',
       'Home_blocked', 'Home_faceOffWinPercentage', 'Home_giveaways',
       'Home_goals', 'Home_hits', 'Home_losses', 'Home_pim',
       'Home_powerPlayGoals', 'Home_powerPlayOpportunities',
       'Home_powerPlayPercentage', 'Home_shots', 'Home_takeaways', 'Home_wins',
       'Loss_Diff', 'OT_Diff', 'PIM_Diff', 'PowerPlayGoals_Diff', 'Shots_Diff',
       'Takeaways_Diff', 'Unnamed: 0', 'Win_Diff', 'Winner_binary'],
      dtype='object')
   Away_OT  Away_blocked  Away_faceOffWinPercentage  Away_giveaways  \
0        3             3                      0.412               4   
1        5             3                      0.588  

In [27]:
X = df2.iloc[:,:-1]
y = df2.iloc[:,-1]

In [28]:
X

,Away_OT,Away_blocked,Away_faceOffWinPercentage,Away_giveaways,Away_goals,Away_hits,Away_losses,Away_pim,Away_powerPlayGoals,Away_powerPlayOpportunities,...,Home_takeaways,Home_wins,Loss_Diff,OT_Diff,PIM_Diff,PowerPlayGoals_Diff,Shots_Diff,Takeaways_Diff,Unnamed: 0,Win_Diff
0,3,3,0.412,4,0,8,21,0,0.0,0.0,...,4,28,-12,2,0,0.0,-3,1,2019020653,9
1,5,3,0.588,0,2,9,15,6,0.0,0.0,...,1,25,-4,0,-4,0.0,-1,0,2019020654,4
2,4,2,0.450,2,2,6,13,2,0.0,1.0,...,7,24,2,-2,0,0.0,8,4,2019020655,1
3,5,7,0.611,0,2,4,17,2,1.0,2.0,...,3,20,0,0,2,0.0,-5,3,2019020656,-1
4,3,7,0.556,3,2,12,29,4,1.0,2.0,...,0,18,-11,3,0,-1.0,4,-4,2019020657,8
5,6,6,0.476,6,1,10,15,7,0.0,0.0,...,2,16,5,-1,0,0.0,17,1,2019020658,-3
0,5,1,0.381,0,1,11,17,0,0.0,2.0,...,5,24,-3,0,4,0.0,-6,4,2019020659,2
1,4,3,0.500,4,0,11,16,0,0.0,1.0,...,3,18,1,3,2,0.0,1,2,2019020660,-4
2,4,3,0.667,3,0,9,13,2,0.0,1.0,...,5,25,-1,-1,0,0.0,10,3,2019020661,0
3,8,6,0.533,4,0,7,15,0,0.0,1.0,...,4,17,7,-4,2,0.0,6,4,2019020662,-2


In [29]:
y

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
5    1.0
0    1.0
1    1.0
2    0.0
3    1.0
Name: Winner_binary, dtype: float64

In [30]:
clf = LogisticRegression(random_state=0, solver = "lbfgs").fit(X, y)

In [31]:
clf.score(X, y)

0.7

In [ ]:
team_name = input("Enter your team: ")
print(team_name)

In [ ]:
with urllib.request.urlopen("https://statsapi.web.nhl.com/api/v1/teams") as url:
        data = json.loads(url.read().decode())
team_dict = {}
for i in range(len(data["teams"])):
    team_dict[data['teams'][i]["abbreviation"]] = data['teams'][i]["id"]
team_id = team_dict[team_name]
print(team_id)

In [ ]:
with urllib.request.urlopen(f"https://statsapi.web.nhl.com/api/v1/teams/{team_id}?expand=team.schedule.next") as url:
        data = json.loads(url.read().decode())
games_id_list = data["teams"][0]["nextGameSchedule"]["dates"][0]["games"][0]["gamePk"]
print(games_id_list)

In [ ]:
games_id_list = [2019020657]

In [ ]:
#Extract the win, loss, OT records for each team playing.
def _team_records(game_id):  
    """This function is used to extract the team records for the teams playing today."""
    game_link = f"https://statsapi.web.nhl.com/api/v1/game/{game_id}/feed/live"
    with urllib.request.urlopen(game_link) as url:
        data = json.loads(url.read().decode())
    away_team_id = data["gameData"]["teams"]["away"]["id"]
    home_team_id = data["gameData"]["teams"]["home"]["id"]
    for team_id in ([away_team_id] + [home_team_id]):
        team_link = f"https://statsapi.web.nhl.com/api/v1/schedule?teamId={team_id}"
        with urllib.request.urlopen(team_link) as url:
            team_id_data = json.loads(url.read().decode())
        home_record = list(team_id_data["dates"][0]["games"][0]["teams"]["home"]["leagueRecord"].values())[:3]
        away_record = list(team_id_data["dates"][0]["games"][0]["teams"]["away"]["leagueRecord"].values())[:3]
    return(home_record + away_record)

In [ ]:
def differences(df):
    """Used to calculate the feature differences between the home and away teams, and to
    convert the percentage features to numeric."""
    df["Win_Diff"] = df["Home_wins"] - df["Away_wins"]
    df["Loss_Diff"] = df["Home_losses"] - df["Away_losses"]
    df["OT_Diff"] = df["Home_OT"] - df["Away_OT"]
    df["Goals_Diff"] = df["Home_goals"] - df["Away_goals"]
    df["Shots_Diff"] = df["Home_shots"] - df["Away_shots"]
    df["Blocked_Diff"] = df["Home_blocked"] - df["Away_blocked"]
    df["PIM_Diff"] = df["Home_pim"] - df["Away_pim"]
    df["PowerPlayGoals_Diff"] = df["Home_powerPlayGoals"] - df["Away_powerPlayGoals"]
    df["Takeaways_Diff"] = df["Home_takeaways"] - df["Away_takeaways"]
    df["Giveaways_Diff"] = df["Home_giveaways"] - df["Away_giveaways"]
    df["Hits_Diff"] = df["Home_hits"] - df["Away_hits"]
    df["Home_powerPlayPercentage"] = pd.to_numeric(df["Home_powerPlayPercentage"])/100
    df["Away_powerPlayPercentage"] = pd.to_numeric(df["Away_powerPlayPercentage"])/100
    df["Home_faceOffWinPercentage"] = pd.to_numeric(df["Home_faceOffWinPercentage"])/100
    df["Away_faceOffWinPercentage"] = pd.to_numeric(df["Away_faceOffWinPercentage"])/100
    return(df)

In [ ]:
game_counter = 0
df = pd.DataFrame(columns = [i for i in range(30)])
for game_id in games_id_list:
    Period = 0 
    Start_time_counter = 0
    while Period < 1:
        game_link = f"https://statsapi.web.nhl.com/api/v1/game/{game_id}/feed/live"
        with urllib.request.urlopen(game_link) as url:
            data = json.loads(url.read().decode())
        if data["liveData"]["linescore"]["currentPeriod"] == 1 and data["liveData"]["linescore"]["currentPeriodTimeRemaining"] == "END": 
            team_record = _team_records(game_id)
            some_columns = ["Home_team", "Away_team", "Home_wins", "Home_losses", "Home_OT", "Away_wins", "Away_losses", "Away_OT"]
            home_team_categories = list(data['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats'].keys())
            away_team_categories = list(data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats'].keys())
            home_team_categories = [f"Home_{i}" for i in home_team_categories]
            away_team_categories = [f"Away_{i}" for i in away_team_categories]
            home_team = [data["gameData"]["teams"]["home"]["triCode"]]
            away_team = [data["gameData"]["teams"]["away"]["triCode"]]
            df.columns = some_columns + home_team_categories + away_team_categories
            away_team_stats = data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
            home_team_stats = data['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']
            home_team_values = list(home_team_stats.values())
            away_team_values = list(away_team_stats.values())
            df.loc[str(game_id)] = home_team + away_team + team_record + home_team_values + away_team_values
            Period = 1
            game_counter += 1
            print("Game ", str(game_counter) ,"/", str(len(games_id_list)), f". ID: {game_id} completed at: ", str(datetime.today().hour), ":", str(datetime.today().minute))
            Start_time_counter += 1
        else:
            Period = 0
            print("Check Point: ", datetime.today().hour, ":", datetime.today().minute)
            time.sleep(300) 

In [ ]:
df_all_features = differences(df)
print(df_all_features)

In [ ]:
df_all_features_copy = df_all_features.copy()
df_all_features = df_all_features.drop(columns = ["Home_team", "Away_team"])
df_all_features.astype(float)
print(clf.predict(df_all_features))
print(clf.predict_proba(df_all_features)) #Predicts that the Hurricanes will win with a 99% probability. I need to tie the
#"0" back to the home team. 
# "0": CAR with probability .997
# "1": DET with probability .999

In [ ]:
if clf.predict(df_all_features) == 0:
    print(df_all_features_copy["Home_team"][0], "predicted to beat", df_all_features_copy["Away_team"][0])
else:
    print(df_all_features_copy["Away_team"][0], "predicted to beat", df_all_features_copy["Home_team"][0], "with a probability of", clf.predict_proba(df_all_features))